In [1]:
%matplotlib inline

In [2]:
import finetune
import importlib
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time
import os
import copy


In [7]:

importlib.reload(finetune)

<module 'finetune' from '/home/matteo/Documents/Studie/Master/2021-2022/REDI/Project/REDI-utwente/src/awesome_GNN/finetune.py'>

In [8]:
num_classes = 196
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 15
finetune.FEATURE_EXTRACT = True
finetune.print_hypers() # shows that params have been modified

classifier input size: None
batch size: 8
num epochs: 15
feature extract: True


In [9]:


# Initialize the model for this run
model_ft = finetune.initialize_model(num_classes, use_pretrained=True)


# Print the model we just instantiated
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# dataloaders_dict = finetune.dataloaders()
finetune.finetune_model(model_ft)

Params to learn:
	 fc.weight
	 fc.bias
Epoch 0/14
----------


In [11]:

# model_ft = model_ft.to(finetune.device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
# params_to_update = model_ft.parameters()
# print("Params to learn:")
# if finetune.FEATURE_EXTRACT:
#     params_to_update = []
#     for name,param in model_ft.named_parameters():
#         if param.requires_grad == True:
#             params_to_update.append(param)
#             print("\t",name)
# else:
#     for name,param in model_ft.named_parameters():
#         if param.requires_grad == True:
#             print("\t",name)
#
# # Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [12]:
# Setup the loss fxn
# criterion = nn.CrossEntropyLoss()
#
# # Train and evaluate
# model_ft, hist = finetune.train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=finetune.NUM_EPOCHS, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 5.2171 Acc: 0.0210
val Loss: 5.5201 Acc: 0.0053

Epoch 1/14
----------
train Loss: 4.7389 Acc: 0.0625
val Loss: 5.7119 Acc: 0.0050

Epoch 2/14
----------
train Loss: 4.4254 Acc: 0.1084
val Loss: 5.8705 Acc: 0.0052

Epoch 3/14
----------
train Loss: 4.1639 Acc: 0.1478
val Loss: 6.0756 Acc: 0.0049

Epoch 4/14
----------
train Loss: 3.9577 Acc: 0.1826
val Loss: 6.2775 Acc: 0.0045

Epoch 5/14
----------
train Loss: 3.7827 Acc: 0.2031
val Loss: 6.4477 Acc: 0.0042

Epoch 6/14
----------
train Loss: 3.6648 Acc: 0.2257
val Loss: 6.6009 Acc: 0.0057

Epoch 7/14
----------
train Loss: 3.5296 Acc: 0.2458
val Loss: 6.8010 Acc: 0.0052

Epoch 8/14
----------
train Loss: 3.4518 Acc: 0.2676
val Loss: 6.9587 Acc: 0.0058

Epoch 9/14
----------
train Loss: 3.3713 Acc: 0.2704
val Loss: 7.0086 Acc: 0.0051

Epoch 10/14
----------
train Loss: 3.2943 Acc: 0.2883
val Loss: 7.1767 Acc: 0.0053

Epoch 11/14
----------
train Loss: 3.2170 Acc: 0.3022
val Loss: 7.3165 Acc: 0.0055

Ep